Importation des données

In [22]:
import mlflow
mlflow.autolog()
import pandas as pd
ct = 'QueryResults (1).csv'
df = pd.read_csv(ct)
df

2024/01/07 20:50:08 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


,Id,Title,Body,Tags,Score,ViewCount,AnswerCount
0,20485075,How do I define multiple conditions on a join ...,"<p>Ok, lets say I have order and items tables....",<java><jpa><playframework><playframework-2.0><...,1,1984,2
1,20485079,What is the equivalent of a C# gridview in jav...,"<p>I am trying to learn java, I am working on ...",<c#><java><asp.net><swing><gridview>,2,12333,1
2,20485143,"Listview cursor adapter, always get first item...",<p>I have this piece of code for listview usin...,<android><listview><android-listview><simplecu...,1,1075,2
3,20485144,OpenSSL RSA-2048 unencrypted block is longer t...,<p>I am using the OpenSSL library in order to ...,<c><encryption><cryptography><openssl><rsa>,0,527,1
4,20485176,Returning char into main method,<p>The goal of my program is kind of like whee...,<java><eclipse><string><methods><char>,0,1278,3
...,...,...,...,...,...,...,...
49995,22814994,StructureMap use specific instance of type per...,"<p>Is there a way, using StructureMap (the Dep...",<c#><asp.net><dependency-injection><inversion-...,0,369,2
49996,22815009,Add a reference column migration in Rails 4,<p>A user has many uploads. I want to add a co...,<ruby-on-rails><ruby-on-rails-4><foreign-keys>...,368,397557,8
49997,22815010,Hibernate mapping with group by clause?,<p>I have <code>@OneToMany</code> mapping in a...,<java><sql><hibernate><group-by><hibernate-map...,0,1755,1
49998,22815022,java.lang.NoClassDefFoundError: android.suppor...,<p>I have a class that extends <code>android.s...,<java><android><eclipse><android-fragments><an...,0,1441,1


Filtre des tags

In [23]:
df['Tags'] = df['Tags'].str.replace('><', ',').str.replace('<', '').str.replace('>', '')

from collections import Counter
# Fréquence des mots dans 'Tags'
all_tags = df['Tags'].str.split(',').explode().tolist()
tag_counts = Counter(all_tags)

# Trouver les 50 mots les plus fréquents
top_50_tags = [tag for tag, _ in tag_counts.most_common(50)]

# Filtrer les listes de mots pour ne contenir que les 50 mots les plus fréquents
def filter_tags(tags):
    return [tag for tag in tags if tag in top_50_tags]

df['Tags'] = df['Tags'].str.split(',').apply(filter_tags)

# Filtrer les lignes où 'Tags' est vide après le filtrage
df = df[df['Tags'].apply(len) > 0]

In [24]:
df['Tags']

0                     [java]
1        [c#, java, asp.net]
2                  [android]
3                        [c]
4             [java, string]
                ...         
49995          [c#, asp.net]
49996        [ruby-on-rails]
49997            [java, sql]
49998        [java, android]
49999      [sql-server, sql]
Name: Tags, Length: 43428, dtype: object

Nettoyage du texte

In [25]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from bs4 import BeautifulSoup

# Nettoyage
def strip_html_bs(text):
    """nettoyage des balises html"""
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

def tokenizer_fct(sentence):
    """Division de mots en texte + suppression de certains caractères"""
    sentence_clean = sentence.replace('-', ' ').replace('+', ' ').replace('/', ' ').replace('#', ' ')
    word_tokens = word_tokenize(sentence_clean)
    return word_tokens

stop_w = list(set(stopwords.words('english'))) + ['[', ']', ',', '.', ':', '?', '(', ')','<','>','~']

def stop_word_filter_fct(list_words):
    """Suppression de mots sans information+ ponctuations"""
    filtered_w = [w for w in list_words if not w in stop_w]
    filtered_w2 = [w for w in filtered_w if len(w) > 2]
    return filtered_w2

def lower_start_fct(list_words):
    """Conversion en lettres minuscules et suppression de préfixes indésirables"""
    lw = [w.lower() for w in list_words if (not w.startswith("@")) and (not w.startswith("#")) and (not w.startswith("http"))]
    return lw

def lemma_fct(list_words):
    """lemmatisation"""
    lemmatizer = WordNetLemmatizer()
    lem_w = [lemmatizer.lemmatize(w) for w in list_words]
    return lem_w

def transform_bow_fct(desc_text):
    """fonction de transformation"""
    text_stripped = strip_html_bs(desc_text)
    word_tokens = tokenizer_fct(text_stripped)
    sw = stop_word_filter_fct(word_tokens)
    lw = lower_start_fct(sw)
    lem = lemma_fct(lw) 
    transf_desc_text = ' '.join(lem)
    return transf_desc_text
    
# Prétraitement
df['Cleaned_Body'] = df['Body'].apply(transform_bow_fct)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/orphila_adjovi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/orphila_adjovi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/orphila_adjovi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/var/folders/_l/cjjst_c50lx2nkkr280zbp440000gp/T/ipykernel_52251/874419303.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cleaned_Body'] = df['Body'].apply(transform_bow_fct)


Encoding TF - IDF

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_array = tfidf_vectorizer.fit_transform(df['Cleaned_Body']).toarray()

# Liste des termes correspondant aux colonnes de la matrice
feature_names = tfidf_vectorizer.get_feature_names_out()
# Scores de chaque mot
tfidf_df = pd.DataFrame(tfidf_array, columns=feature_names)

df.reset_index(drop=True, inplace=True)

Création de modèles LDA avec 50 sujets

In [30]:
import gensim
import pyLDAvis
import pyLDAvis.gensim as gensimvis

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

corpus = df['Cleaned_Body']
X = tfidf_df

tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)
terms = tfidf_vectorizer.get_feature_names_out()

# Convertion de la matrice TF-IDF en une représentation sparse de Gensim
corpus_tfidf_gensim = gensim.matutils.Sparse2Corpus(tfidf_matrix.T)

# Création d'un dictionnaire Gensim à partir de TF-IDF
dictionary = gensim.corpora.Dictionary.from_corpus(corpus_tfidf_gensim, id2word=dict((i, s) for i, s in enumerate(terms)))

# Entraînement du modèle LDA avec Gensim et sklearn
num_topics = 50
lda_model_gensim= gensim.models.LdaModel(corpus=corpus_tfidf_gensim, id2word=dictionary, num_topics=num_topics, random_state=42)
lda_model_sklearn = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda_model_sklearn.fit(X)

# Convertir les tags en ensembles
set_tags = df['Tags'].apply(set)

# Obtention des n sujets les plus probables pour chaque ligne de tfidf_df, où n est le nombre de tags
top_topics_gensim = lda_model_gensim.get_document_topics(corpus_tfidf_gensim)
top_topics_gensim = [sorted(topics, key=lambda x: x[1], reverse=True)[:len(tags)] for topics, tags in zip(top_topics_gensim,set_tags)]

top_topics_sklearn = lda_model_sklearn.transform(X).argsort(axis=1)[:, ::-1]
top_topics_sklearn = [topics[:len(tags)] for topics, tags in zip(top_topics_sklearn, set_tags)]

# Création d'une colonne 'lda_predict' avec la liste des sujets les plus probables
df['lda_predict_gensim'] = [topics for topics in top_topics_gensim]
df['lda_predict_gensim'] = df['lda_predict_gensim'].apply(lambda x: [topic[0] for topic in x])
df['lda_predict_sklearn'] = [topics for topics in top_topics_sklearn]

# Enregistrement du modèle Gensim dans MLflow
with mlflow.start_run(run_name='unsupervised_gensim_final_lem') as run:
    mlflow.sklearn.log_model(
        sk_model=lda_model_gensim,
        artifact_path='lda_gensim_final',
        registered_model_name="unsupervised_gensim_final",
    )

# Enregistrement du modèle sklearn dans MLflow
with mlflow.start_run(run_name='unsupervised_sklearn_final_lem') as run:
    mlflow.sklearn.log_model(
        sk_model=lda_model_sklearn,
        artifact_path='lda_sklearn_final',
        registered_model_name="unsupervised_sklearn_final",
    )


2024/01/07 21:08:33 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '33a14861634f485280aa087387ce393e', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/01/07 21:08:34 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/.venv/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."
2024/01/07 21:09:55 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2024/01/07 21:09:55 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2024/01/07 21:10:17 WARNING mlflow.utils.autologging_utils: MLflow autologging encounter

Visualisation avec LDAvis

In [31]:
vis_data = gensimvis.prepare(lda_model_gensim, corpus_tfidf_gensim, dictionary)
pyLDAvis.display(vis_data)

In [37]:
df[['Tags','lda_predict_gensim']]

,Tags,lda_predict_gensim
0,[java],[6]
1,"[c#, java, asp.net]","[6, 22, 14]"
2,[android],[7]
3,[c],[19]
4,"[java, string]","[35, 31]"
...,...,...
43423,"[c#, asp.net]","[47, 41]"
43424,[ruby-on-rails],[10]
43425,"[java, sql]","[48, 20]"
43426,"[java, android]","[48, 36]"


Retrouver les tags correspondant aux sujets

In [38]:
from collections import defaultdict

# Création d'un dictionnaire pour compter les correspondances
tag_counts = defaultdict(lambda: defaultdict(int))

# Parcours des lignes pour compter les correspondances entre les tags et les numéros
for index, row in df.iterrows():
    tags = row['Tags']
    lda_topics = row['lda_predict_sklearn']

    # Comptage des correspondances entre les tags et les numéros
    for topic in lda_topics:
        for tag in tags:
            tag_counts[topic][tag] += 1

# Convertion du dictionnaire en DataFrame
results = pd.DataFrame(tag_counts).T
results.fillna(0, inplace=True)
results = results.rename_axis('LDA Topic').reset_index()

In [39]:

topic_to_tag = {}
tag_to_topic = {}

# Parcourir les lignes du DataFrame et associer chaque sujet au tag le plus fréquent
for _, row in results.iterrows():
    topic = row['LDA Topic']
    tags_except_lda = row.drop('LDA Topic')
    
    # Vérifier si le tag a déjà été associé à un autre sujet
    if topic not in topic_to_tag and not tags_except_lda.empty:
        # Si le tag n'a pas encore été associé à un sujet
        most_common_tag = tags_except_lda.idxmax()
        
        # Vérifier si le tag a déjà été associé à un autre sujet
        while most_common_tag in tag_to_topic:
            # Si le tag a déjà été associé, trouver le prochain tag le plus fréquent
            tags_except_lda = tags_except_lda.drop(most_common_tag)
            if tags_except_lda.empty:
                # Si tous les tags ont déjà été associés, sortir de la boucle
                break
            most_common_tag = tags_except_lda.idxmax()
        
        # Associer le sujet au tag le plus fréquent
        topic_to_tag[topic] = most_common_tag
        tag_to_topic[most_common_tag] = topic


# Remplacer les numéros dans df['lda_predict'] par les tags
df['lda_predict_sklearn'] = df['lda_predict_sklearn'].apply(lambda topics: [topic_to_tag.get(t, 'Unknown') for t in topics])
df['lda_predict_gensim'] = df['lda_predict_gensim'].apply(lambda topics: [topic_to_tag.get(t, 'Unknown') for t in topics])

/var/folders/_l/cjjst_c50lx2nkkr280zbp440000gp/T/ipykernel_52251/1662499294.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lda_predict_sklearn'] = df['lda_predict_sklearn'].apply(lambda topics: [topic_to_tag.get(t, 'Unknown') for t in topics])
/var/folders/_l/cjjst_c50lx2nkkr280zbp440000gp/T/ipykernel_52251/1662499294.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lda_predict_gensim'] = df['lda_predict_gensim'].apply(lambda topics: [topic_to_tag.get(t, 'Unknown') for t in topics])


In [40]:
import json
json_data = json.dumps(topic_to_tag)
with open("topic_to_tag.json", "w") as file:
    file.write(json_data)
mlflow.log_artifact("topic_to_tag.json")

import os
os.remove("topic_to_tag.json")

In [41]:
df[['Tags','lda_predict_gensim']]

,Tags,lda_predict_gensim
0,[java],[php]
1,"[c#, java, asp.net]","[php, c#, image]"
2,[android],[spring]
3,[c],[visual-studio]
4,"[java, string]","[spring-boot, objective-c]"
...,...,...
43423,"[c#, asp.net]","[java, regex]"
43424,[ruby-on-rails],[ruby-on-rails]
43425,"[java, sql]","[c++, asp.net-mvc]"
43426,"[java, android]","[c++, angularjs]"


Scoring

In [45]:
from sklearn.metrics import jaccard_score
from sklearn.preprocessing import MultiLabelBinarizer

# Convertir les listes d'identifiants en représentation binaire
mlb = MultiLabelBinarizer()
y_true = mlb.fit_transform(df['Tags'])

# Convertir les listes d'identifiants pour lda_predict_gensim
y_pred_gensim = mlb.transform(df['lda_predict_gensim'])
# Calculer le Jaccard score pour lda_predict_gensim
jaccard_score_gensim = round(jaccard_score(y_true, y_pred_gensim, average='micro', zero_division=1),2)

# Convertir les listes d'identifiants pour lda_predict_sklearn
y_pred_sklearn = mlb.transform(df['lda_predict_sklearn'])
# Calculer le Jaccard score pour lda_predict_sklearn
jaccard_score_sklearn = round(jaccard_score(y_true, y_pred_sklearn, average='micro', zero_division=1),2)

# Afficher les scores Jaccard
print("Jaccard Score pour lda_predict_gensim:", jaccard_score_gensim)
print("Jaccard Score pour lda_predict_sklearn:", jaccard_score_sklearn)


Jaccard Score pour lda_predict_gensim: 0.05
Jaccard Score pour lda_predict_sklearn: 0.12


In [46]:
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel

# Transformer le corpus TF-IDF en une liste de listes de mots
tokenized_data = [text.split() for text in df['Cleaned_Body']]

# Créer un dictionnaire Gensim à partir de vos données tokenizées
dico = Dictionary(tokenized_data)

# Calculer la cohérence
coherence_model = CoherenceModel(model=lda_model_gensim, texts=tokenized_data, dictionary=dico, coherence='c_v')
coherence_score = coherence_model.get_coherence()
print("Coherence Score =",round(coherence_score,2))

perplexity = lda_model_sklearn.perplexity(tfidf_df)  
print("Perplexity= " ,round(perplexity,0))

Coherence Score = 0.42
Perplexity=  2929.0


Tentative d'utiliser un PCA pour réduire la dimension

In [49]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

pca = PCA(n_components=5)
tfidf_pca = pca.fit_transform(tfidf_df)
df_pca = pd.DataFrame(data=tfidf_pca, columns=['PC1', 'PC2','PC3','PC4','PC5'])
#print(df_pca.head())


df_pca_scaled = pd.DataFrame(scaler.fit_transform(df_pca), columns=df_pca.columns)
X = df_pca_scaled
with mlflow.start_run(run_name='unsupervised_PCA_final') as run:
    lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)
    lda_model.fit(X)
    mlflow.sklearn.log_model(
        sk_model=lda_model,
        artifact_path="model_PCA",
        registered_model_name="unsupervised_PCA_final",
    )
# Obtenir les sujets les plus probables pour chaque ligne
top_topics= lda_model.transform(X).argsort(axis=1)[:, ::-1]
top_topics = [topics[:len(tags)] for topics, tags in zip(top_topics, set_tags)]

# Créer une colonne 'lda_predict' avec la liste des sujets les plus probables
df['lda_predict'] = [topics for topics in top_topics]

# Calcul du score Jaccard
mlb = MultiLabelBinarizer()
y_true = mlb.fit_transform(df['Tags'])
y_pred = mlb.transform(df['lda_predict'])
jaccard_score_pca = jaccard_score(y_true, y_pred, average='micro', zero_division=1)

# Afficher le score Jaccard
print("Jaccard Score pour lda_predict après PCA:", jaccard_score_pca)


perplexity = lda_model.perplexity(X)
print("Perplexity= " ,round(perplexity,0))

2024/01/07 21:32:36 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'ba3cd8c85d6247d79cd684672ee105cd', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/01/07 21:32:36 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/.venv/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."
2024/01/07 21:32:45 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2024/01/07 21:32:45 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2024/01/07 21:32:47 WARNING mlflow.utils.autologging_utils: MLflow autologging encounter

Jaccard Score pour lda_predict après PCA: 0.0
Perplexity=  132679884405.0
